In [1]:
# Import data science environment.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import scipy
import sklearn
from sklearn import metrics
from sklearn.cluster import KMeans, MeanShift, SpectralClustering, AffinityPropagation, estimate_bandwidth
from sklearn.preprocessing import normalize
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format
sns.set_style('white')

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(
    action='ignore',
    module='scipy',
    message='internal gelsd'
)

In [2]:
boston2013 = pd.read_csv('https://raw.githubusercontent.com/llimllib/bostonmarathon/master/results/2013/results.csv')
boston2013.head()

,25k,age,name,division,10k,gender,half,official,bib,ctz,...,overall,pace,state,30k,5k,genderdiv,20k,35k,city,40k
0,49.87,28,"Cassidy, Josh R.",9,18.18,M,40.93,90.900,W1,NaN,...,9,3.470,ON,62.07,8.90,9,38.80,74.73,Toronto,85.55
1,77.27,30,"Korir, Wesley",5,30.90,M,64.90,132.500,1,NaN,...,5,5.070,NaN,92.97,15.90,5,61.52,108.78,Kenya,124.77
2,77.23,23,"Desisa, Lelisa",1,30.90,M,64.92,130.370,2,NaN,...,1,4.980,NaN,92.72,15.93,1,61.53,108.68,Ambo,123.78
3,50.50,32,"Fearnley, Kurt H.",5,18.73,M,42.00,88.430,W2,NaN,...,5,3.380,NaN,61.35,8.98,5,39.88,73.00,Hamilton,83.43
4,48.75,39,"Hokinoue, Kota",3,18.18,M,40.57,87.220,W3,NaN,...,3,3.330,NaN,59.92,8.92,3,38.55,71.68,Iizuka,81.88


In [3]:
boston2013.describe()

,age,division,official,overall,pace,genderdiv
count,16164.000,16164.000,16164.000,16164.000,16164.000,16164.000
mean,41.638,1100.967,208.159,8429.373,7.947,4351.685
std,10.351,942.115,23.744,5052.024,0.906,2772.398
min,18.000,1.000,85.530,1.000,3.270,1.000
25%,34.000,363.000,191.727,4061.750,7.320,2032.750
50%,42.000,842.000,209.225,8247.500,7.980,4113.500
75%,49.000,1560.000,225.230,12662.250,8.600,6316.000
max,80.000,3834.000,284.230,17598.000,10.850,10648.000


In [4]:
# Check dtypes.
boston2013.dtypes

25k           object
age            int64
name          object
division       int64
10k           object
gender        object
half          object
official     float64
bib           object
ctz           object
country       object
overall        int64
pace         float64
state         object
30k           object
5k            object
genderdiv      int64
20k           object
35k           object
city          object
40k           object
dtype: object

## Data cleaning

In [5]:
# Drop columns that are not relevant.
boston2013 = boston2013.drop(['name', 'division', 'bib', 'ctz', 'country', 'state', 'city'], 1)

In [6]:
# Rearrange columns.
boston2013 = boston2013[['age', '5k', '10k', '20k', 'half', '25k', '30k',
                        '35k', '40k', 'official', 'pace']]

In [7]:
# Drop other missing data.
boston2013 = boston2013.dropna()

In [8]:
# Remove hyphens from missing values.
boston2013 = boston2013.replace(to_replace='-',value='0')

In [9]:
# Convert race times to numeric values.
columns = boston2013[['5k','10k','20k','25k','half','30k','35k','40k','age','official','pace']]

for column in columns: 
    boston2013[column] = pd.to_numeric(boston2013[column], errors='coerce')

In [10]:
# See data frame format.
boston2013.head()

,age,5k,10k,20k,half,25k,30k,35k,40k,official,pace
0,28,8.900,18.180,38.800,40.930,49.870,62.070,74.730,85.550,90.900,3.470
1,30,15.900,30.900,61.520,64.900,77.270,92.970,108.780,124.770,132.500,5.070
2,23,15.930,30.900,61.530,64.920,77.230,92.720,108.680,123.780,130.370,4.980
3,32,8.980,18.730,39.880,42.000,50.500,61.350,73.000,83.430,88.430,3.380
4,39,8.920,18.180,38.550,40.570,48.750,59.920,71.680,81.880,87.220,3.330


In [11]:
# View measures of central tendency.
boston2013.describe()

,age,5k,10k,20k,half,25k,30k,35k,40k,official,pace
count,16164.000,16164.000,16164.000,16164.000,16164.000,16164.000,16164.000,16164.000,16164.000,16164.000,16164.000
mean,41.638,23.329,46.655,93.953,99.133,118.036,143.425,169.750,196.352,208.159,7.947
std,10.351,2.879,5.254,10.410,10.965,13.423,16.345,19.928,23.230,23.744,0.906
min,18.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,85.530,3.270
25%,34.000,21.700,43.200,86.765,91.530,108.900,132.170,156.345,180.900,191.727,7.320
50%,42.000,23.630,47.170,94.870,100.080,119.150,144.700,171.210,197.780,209.225,7.980
75%,49.000,25.180,50.280,101.270,106.850,127.320,154.780,183.570,212.600,225.230,8.600
max,80.000,33.220,66.680,131.720,138.670,163.620,195.870,229.500,268.400,284.230,10.850


In [12]:
# Double-check dtypes.
boston2013.dtypes

age           int64
5k          float64
10k         float64
20k         float64
half        float64
25k         float64
30k         float64
35k         float64
40k         float64
official    float64
pace        float64
dtype: object

In [13]:
# Set and normalize X.
X = boston2013.loc[:, ['5k', '10k', '20k', 'half', '25k', '30k', '35k', '40k', 'official']]
X_norm = normalize(X)

## K-Means

In [14]:
# Create and run k-means model.
for k in np.arange(2, 8, 1):
    model = KMeans(n_clusters=k, random_state=15).fit(X_norm)
    labels = model.labels_
    silhouette = metrics.silhouette_score(X_norm, labels, metric='euclidean')
    print('K: {}, silhouette: {}'.format(k, silhouette))

K: 2, silhouette: 0.9789411435901354
K: 3, silhouette: 0.5634915374561319
K: 4, silhouette: 0.5646407072166075
K: 5, silhouette: 0.5682025200221598
K: 6, silhouette: 0.4782449973090199
K: 7, silhouette: 0.48036144452076585


## Mean Shift

In [15]:
# Set bandwidth estimator.
bandwidth = estimate_bandwidth(X_norm, quantile=0.2, n_samples=500)

# Declare and fit model.
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(X_norm)

# Get labels, number of clusters, similarity score.
labels = ms.labels_
n_clusters = len(np.unique(labels))
silhouette = metrics.silhouette_score(X_norm, labels, metric='euclidean')
print('K: {}, silhouette: {}'.format(n_clusters, silhouette))

K: 119, silhouette: 0.15808572817557895


## Spectral Clustering

In [16]:
# Run spectral clustering over range of k values.
for k in np.arange(2, 6, 1):
    sc = SpectralClustering(n_clusters=k, random_state=15).fit(X_norm)
    labels = model.labels_
    silhouette = metrics.silhouette_score(X_norm, labels, metric='euclidean')
    print('K: {}, silhouette: {}'.format(k, silhouette))

K: 2, silhouette: 0.48036144452076585
K: 3, silhouette: 0.48036144452076585
K: 4, silhouette: 0.48036144452076585
K: 5, silhouette: 0.48036144452076585


## Affinity Propagation

In [17]:
# Run and fit model.
af = AffinityPropagation(max_iter=20).fit(X_norm)

# Get number of clusters.
cluster_centers_indices = af.cluster_centers_indices_
n_clusters = len(cluster_centers_indices)
print('K: {}'.format(n_clusters))

K: 3855
